In [ ]:
# Import the needed libraries
import os
import pandas as pd
import numpy as np
import math

import torchvision
import torchvision.transforms as T

import torch
from torch.utils.data import DataLoader

from lib.TowerDataset2 import TowerDataset

from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


# Print PyTorch version
print("PyTorch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

# Print CUDA version used by PyTorch
print("CUDA version:", torch.version.cuda)

# To check if CUDA is available in your PyTorch installation
if torch.cuda.is_available():
    print("CUDA is available. GPU Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")

# Nvidia settings
# !nvidia-smi

In [ ]:
# Define data transform
tfm = T.ToTensor()

# Define datasets
ds = TowerDataset('data/', tfm)


model = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# Define your custom sizes
min_size = 150  # The minimum size of the image during training/testing
max_size = 150  # The maximum size of the image during training/testing

# Adjust the min_size and max_size of the transform
model.transform.min_size = (min_size,)
model.transform.max_size = max_size

# Modify the classifier to fit the number of classes
num_classes = 10  # Your number of classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Move model to the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

from lib.detection.utils import collate_fn
from torch.utils.data import random_split

#split the dataset in training and test set
number = len(ds)
train_number = math.ceil(number * 0.7)
dataset_train, dataset_val = random_split(ds, [train_number, number-train_number])

# define training and validation data loaders
data_loader_train = DataLoader(
    dataset_train, batch_size=2, shuffle=True, num_workers=8,
    collate_fn=collate_fn
)

data_loader_val = DataLoader(
    dataset_val, batch_size=2, shuffle=False, num_workers=6,
    collate_fn=collate_fn
)


from torch.optim.lr_scheduler import StepLR
from torch.optim import SGD

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.0001, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = StepLR(optimizer,
                      step_size=10,
                      gamma=0.1)


from lib.detection.coco_utils import get_coco_api_from_dataset
coco = get_coco_api_from_dataset(data_loader_val.dataset)

In [ ]:
from lib.detection.engine import train_one_epoch, evaluate
from torch.utils.tensorboard import SummaryWriter

print('start')

num_epochs = 30

writer = SummaryWriter()

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader_train,
                    device, epoch, writer=writer)

    # update the learning rate
    print(f'Epoch {epoch} training done')
    lr_scheduler.step()
    
    # evaluate on the validation dataset
    evaluate(model, data_loader_val, device, epoch, coco, writer=writer)
    

In [ ]:
torch.save(model.state_dict(), 'model/model_weights.pth')